# ECG Classification

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from biosppy.signals.ecg import extract_heartbeats
from biosppy.signals.ecg import ecg
import pdb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import math
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter
from sklearn.cluster import KMeans
from scipy.stats import norm
from tqdm import trange, tqdm

In [2]:
X = pd.read_csv('X_train.csv')
y = pd.read_csv('y_train.csv')

X = X.drop(['id'],axis=1)
y = y.drop(['id'],axis=1)

In [3]:
X = X.fillna(0.0)
X.at[5116,'x17978'] = 0.0
X = X.astype(float)

## Methods

In [4]:
def onlyVar(signal,show=False):
    signal = removeStartEnd(signal)
    ecgResults = ecg(signal,sampling_rate=300, show=False)
    templates = np.swapaxes(ecgResults[4],1,0)
    
    # Summed variance over whole tamplate scaled by Median peak amplitude
    peakAmplitudes = []
    for k in ecgResults[2]:
        peakAmplitudes.append(ecgResults[1][k])
    amplitudeMedian = np.median(np.array(peakAmplitudes))
    scaledTemplates = templates/amplitudeMedian
    sumVar = np.sum(scaledTemplates.var(axis=1))
    
    return sumVar

In [5]:
def removeStartEnd(signal):
    end = np.count_nonzero(signal)
    start = 0
    if (type(signal) == type(np.array([2]))):
        return signal[start+300:end-300]
    return signal[start+300:end-300].to_numpy()

In [6]:
def getQs(ecgResults):
    '''
    :return qsTime: timepoints of Q in the whole time
            qsValue: value of Q's amplitude
    '''
    
    #pdb.set_trace()
    qsTime = []
    qsValue = []
    
    for peak in ecgResults[2]:
        # Heartbeat is at timestamp 60 (=time 0)
        numChances = 2
        currValue = ecgResults[1][peak]
        nextValue = ecgResults[1][peak]
        foundMinimum = ecgResults[1][peak]
        foundMinimumIndex = peak
        
        i = peak-1
        while (nextValue <= currValue or numChances >= 0):
            # update
            currValue = nextValue
            nextValue = ecgResults[1][i]
            if (foundMinimum >= nextValue):
                foundMinimum = nextValue
                foundMinimumIndex = i
            i = i-1
            if (currValue < nextValue):
                numChances -= 1
            
        qsTime.append(foundMinimumIndex)
        qsValue.append(foundMinimum)
        
    return [qsTime, qsValue]

In [7]:
def getSs(ecgResults):
    sTime = []
    sValue = []
    
    for peak in ecgResults[2]:
        # Heartbeat is at timestamp 60 (=time 0)
        numChances = 2
        currValue = ecgResults[1][peak]
        nextValue =ecgResults[1][peak]
        foundMinimum = ecgResults[1][peak]
        foundMinimumIndex = peak
        
        i = peak+1
        while (nextValue <= currValue or numChances >= 0):
            # update
            currValue = nextValue
            nextValue = ecgResults[1][i]
            if (foundMinimum >= nextValue):
                foundMinimum = nextValue
                foundMinimumIndex = i
            i = i+1
            if (currValue < nextValue):
                numChances -= 1
            
        sTime.append(foundMinimumIndex)
        sValue.append(foundMinimum)

        
    return [sTime, sValue]

In [8]:
def getAmplitudes(signal,points,show=False):
    amps = []
    for p in points:
        amps.append(signal[p])
    
    if (show):
        plt.plot(signal)
        plt.plot(points,amps)
    return amps

In [9]:
def getQsNormal(signal, peakTimes):
    qsTime = []
    qsValue = []
        
    for peak in peakTimes:
        numChances = 2
        currValue = signal[peak]
        nextValue = signal[peak]
        foundMinimum = signal[peak]
        foundMinimumIndex = peak
        
        i = peak-1
        while (nextValue <= currValue or numChances >= 0):
            if(i==-1):
                foundMinimumIndex = i+1
                foundMinimum = nextValue
                break
            # update
            currValue = nextValue
            nextValue = signal[i]
            if (foundMinimum >= nextValue):
                foundMinimum = nextValue
                foundMinimumIndex = i
            i = i-1
            if (currValue < nextValue):
                numChances -= 1
                
        qsTime.append(foundMinimumIndex)
        qsValue.append(foundMinimum)
        
    return [qsTime, qsValue]

In [10]:
def getSsNormal(signal, peakTimes):
    sTime = []
    sValue = []
    
    for peak in peakTimes:
        # Heartbeat is at timestamp 60 (=time 0)
        numChances = 2
        currValue = signal[peak]
        nextValue = signal[peak]
        foundMinimum = signal[peak]
        foundMinimumIndex = peak
        
        i = peak+1
        while (nextValue <= currValue or numChances >= 0):
            if(i==17979):
                foundMinimumIndex = i-1
                foundMinimum = nextValue
                break
            # update
            currValue = nextValue
            nextValue = signal[i]
            if (foundMinimum >= nextValue):
                foundMinimum = nextValue
                foundMinimumIndex = i
            i = i+1
            if (currValue < nextValue):
                numChances -= 1
            
        sTime.append(foundMinimumIndex)
        sValue.append(foundMinimum)

        
    return [sTime, sValue]

In [11]:
def quartile(arr,percent):
    '''
    param: percent (should be between 0 and 1) how much you want to keep
    '''
    if(type(arr) != type(np.array(arr))):
        arr = np.array(arr)
    length = arr.shape[0]
    if(length < 2):
        return arr
    numOuts = math.ceil(length*(1-percent))
    median = np.median(arr)
    dists = []
    dists2 = []
    for i in range(0,length):
        dists.append(np.absolute(arr[i]-median))
        dists2.append(np.absolute(arr[i]-median))
    dists = np.array(dists)
    dists.sort()
    outs = dists[length-numOuts:length]
    minDistExclusive = np.min(outs)
    res = []
    for i in range(0,length):
        if(dists2[i] < minDistExclusive):
            res.append(arr[i])

    return np.array(res)

In [12]:
def medianvar(templates):
    arr = np.swapaxes(templates,1,0)
    variances = []
    for i in range(0,arr.shape[0]):
        variances.append(np.var(medianGaps(arr[i])))
    varSum = 0
    for var in variances:
        varSum += var
    return varSum

In [13]:
def medianGaps(arr):
    median = np.median(arr)
    gaps = []
    for i in range(0,arr.shape[0]):
        gaps.append(np.absolute(median - arr[i]))
    return np.array(gaps)

In [14]:
def getGaps(arr):
    if(type(arr) != type(np.array(arr))):
        arr = np.array(arr)
    length = arr.shape[0]
    if (length == 2):
        return np.absolute(arr[0]-arr[1])
    gaps = []
    prev = arr[0]
    for i in range(1,length):
        gaps.append(np.absolute(prev-arr[i]))
        prev = arr[i]
    return np.array(gaps)

## Swap To correct Side

In [15]:
invs = []
fract = []
for i in range(0, X.shape[0]):
    var = onlyVar(X.iloc[i])
    varI = onlyVar(X.iloc[i]*(-1))
    if (varI < var):
        invs.append(-1)
        fract.append(var/varI)
    else:
        invs.append(1)
        fract.append(varI/var)

In [16]:
X_corr = X.mul(np.array(invs),axis=0)

## Delete Messfehler

In [17]:
def delMessfehlerMinimum(signal, mul, margin):
    if(type(signal) != type(np.array([1]))):
        signal = signal.to_numpy()
        
    ecgResults = ecg(signal,sampling_rate=300,show=False)
    Rs = ecgResults[2]
    Qs = getQs(ecgResults)
    Ss = getSs(ecgResults)

    #amps = getAmplitudes(signal,Rs, show)
    Samps = getAmplitudes(signal,Ss[0])
    Qamps = getAmplitudes(signal,Qs[0])
    
    mi = min(np.median(Samps),np.median(Qamps),-300)
    margin = margin
    returnSignal = signal

    # Take all number which are siginificantely bigger than the quartile minimum
    timesmi = []
    for i in range(0,signal.shape[0]):
        if (signal[i] < mi*mul):
            timesmi.append(i)

    if (len(timesmi)==0):
        return returnSignal
    
    clusters = []
    cluster = [timesmi[0]]

    for i in range(1,len(timesmi)):
        if (timesmi[i]-timesmi[i-1]>= 100):
            clusters.append(cluster)
            cluster = [timesmi[i]]
        else:
            cluster.append(timesmi[i])
    clusters.append(cluster)
                
    # many clusters: Then probaly class 2, and we only delete culters with more than 30 entries
    corr = 0 # Since we slice the indexes change, so we need to correct them
    if (len(clusters) >= 5):
        amps = []
        for cluster in clusters:
            ampclusters = []
            for el in cluster:
                ampclusters.append(signal[el])
            amps.append(np.min(np.array(ampclusters)))
        median = np.median(np.array(amps))
        for cluster in clusters:
            ampclusters = []
            for el in cluster:
                ampclusters.append(returnSignal[el-corr])
            if (np.min(np.array(ampclusters)) < median*1.2):
                if(cluster[0]-1-margin-corr < 0):
                    continue
                returnSignal = np.concatenate((returnSignal[0:cluster[0]-1-margin-corr],returnSignal[cluster[-1]-corr+margin:]))
                corr += (cluster[-1]-cluster[0])+1+2*margin  # The plus 2*margin becuase we exluce [a-margin,b+margin] instead of [a,b] 
    # Only a few clusters. We assume everything i
    else:
        for cluster in clusters:
            if(cluster[0]-1-margin-corr < 0):
                continue
            returnSignal = np.concatenate((returnSignal[0:cluster[0]-1-margin-corr],returnSignal[cluster[-1]-corr+margin:]))
            corr += (cluster[-1]-cluster[0])+1+2*margin  # The plus 2*margin becuase we exluce [a-margin,b+margin] instead of [a,b] 
    ret = np.zeros(17979)
    for i in range(len(returnSignal)):
        ret[i]=returnSignal[i]
    return ret

In [18]:
def delMessfehlerMaximum(signal, mul, margin):
    if(type(signal) != type(np.array([1]))):
        signal = signal.to_numpy()
    
    ecgResults = ecg(signal,sampling_rate=300,show=False)
    Rs = ecgResults[2]
    Ramps = getAmplitudes(signal,Rs)    

    margin = margin
    ma = np.median(Ramps) # Take median of peaks
    returnSignal = signal

    # Take all number which are siginificantely bigger than the quartile minimum
    timesmi = []
    for i in range(0,signal.shape[0]):
        if (signal[i] > ma*mul):
            timesmi.append(i)
                        
    if (len(timesmi)==0):
        return returnSignal
    
    clusters = []
    cluster = [timesmi[0]]

    for i in range(1,len(timesmi)):
        if (timesmi[i]-timesmi[i-1]>= 100):
            clusters.append(cluster)
            cluster = [timesmi[i]]
        else:
            cluster.append(timesmi[i])
    clusters.append(cluster)
    
    # many clusters: Then probaly class 2, and we only delete culters with more than 30 entries
    corr = 0 # Since we slice the indexes change, so we need to correct them
    if (len(clusters) >= 5):
        amps = []
        for cluster in clusters:
            ampclusters = []
            for el in cluster:
                ampclusters.append(signal[el])
            amps.append(np.max(np.array(ampclusters)))
        median = np.median(np.array(amps))
        for cluster in clusters:
            ampclusters = []
            for el in cluster:
                ampclusters.append(returnSignal[el-corr])
            if (np.max(np.array(ampclusters)) > median*1.2):
                if(cluster[0]-1-margin-corr < 0):
                    continue
                returnSignal = np.concatenate((returnSignal[0:cluster[0]-1-margin-corr],returnSignal[cluster[-1]-corr+margin:]))
                corr += (cluster[-1]-cluster[0])+1+2*margin # The plus 2*margin becuase we exluce [a-margin,b+margin] instead of [a,b] 
    # Only a few clusters. We assume everything i
    else:
        for cluster in clusters:
            if(cluster[0]-1-margin-corr < 0):
                continue
            returnSignal = np.concatenate((returnSignal[0:cluster[0]-1-margin-corr],returnSignal[cluster[-1]-corr+margin:]))
            corr += (cluster[-1]-cluster[0])+1+2*margin  # The plus 2*margin becuase we exluce [a-margin,b+margin] instead of [a,b] 
    ret = np.zeros(17979)
    for i in range(len(returnSignal)):
        ret[i]=returnSignal[i]
    return ret

In [19]:
for i in range(0,X_corr.shape[0]):
    signal = X_corr.iloc[i]
    newsignal = delMessfehlerMinimum(signal,1.7,50)
    newsignal = delMessfehlerMaximum(newsignal,1.7,50)
    X_corr.iloc[i] = newsignal

## Train Features

In [20]:
X_no3 = X_corr[y.y != 3]
y_no3 = y[y.y != 3]

#### Nici Features

In [21]:
def shiftWindow(wSize,signal,show=False):
    ecgResults = ecg(signal,sampling_rate=300, show=show)
    templates = ecgResults[4]

    # Summed variance over whole tamplate scaled by Median peak amplitude
    peakAmplitudes = []
    for k in ecgResults[2]:
        peakAmplitudes.append(ecgResults[1][k])
    amplitudeMedian = np.median(np.array(peakAmplitudes))
    templates = templates/amplitudeMedian

    wSize = min(wSize, templates.shape[0])
    window = templates[0:wSize]
    #print(scaledTemplates.shape)
    winStart = ecgResults[2][0]
    winEnd = ecgResults[2][wSize-1] 
    minVar = medianvar(window)
    
    for i in range(wSize+1, templates.shape[0]):
        if (medianvar(templates[i-wSize:i])<minVar):
            window = templates[i-wSize:i]
            winStart = ecgResults[2][i-1-wSize]
            winEnd = ecgResults[2][i]
            minVar = medianvar(templates[i-wSize:i])
    
    windowMean = np.median(window, axis = 0)
    windowVar = np.var(window, axis = 0)
    
    prTemplates = []
    for t in templates:
        prTemp = []
        for i in range(len(t)):
            prTemp.append(min(1,norm.pdf(x = t[i], loc = windowMean[i],scale = windowVar[i])))
        prTemplates.append(np.sum(prTemp))

    return [minVar, winStart, winEnd, window, prTemplates]

In [22]:
def findP(signal):
    t = np.argmax(signal[0:45])
    #plt.scatter(t,signal[t],color='red')
    return t

def findQ(signal):
    t = np.argmin(signal[25:60])+25
    #plt.scatter(t,signal[t],color='red')
    return t

def findR(signal):
    t = 60
    #plt.scatter(t,signal[t],color='red')
    return t

def findS(signal):
    t = np.argmin(signal[60:110])+60
    #plt.scatter(t,signal[t],color='red')
    return t

def findT(signal):
    t = np.argmax(signal[75:150])+75
    #plt.scatter(t,signal[t],color='red')
    return t

def findTstart(signal, T):
    mi = signal[T]
    miT = T
    k = T
    numChances = 2
    while((mi > signal[k] and signal[k] >= -0.05) or numChances >= 0):
        if(mi > signal[k] and signal[k] >= -0.05):
            mi = signal[k]
            miT = k
        else:
            numChances -= 1
        k -= 1
    #plt.scatter(miT,mi,color='green')
    return miT

def findTend(signal, T):
    t = np.argmin(signal[T:175])+T
    #plt.scatter(t,signal[t],color='green')
    return t

def findSTelbow(signal, S):
    k = S+3
    t = S+3
    numChances = 3
    currA = signal[k]
    nextA = signal[k+1]
    trend = (nextA - currA)*180
    while(trend > 1 or numChances >= 0):
        if (trend < 1):
            numChances -= 1
        else:
            t+= 1
        k+= 1
        trend = (signal[k+1] - signal[k])*180
    #plt.scatter(k,signal[k],color='black')
    return k

def QRTrendChange(signal, Q, R):
    trends=[]
    for k in range(Q,R):
        trends.append((signal[k+1] - signal[k])*180)
    trendCurve = np.array(trends)
    try:
        trendCurve = savgol_filter(trendCurve, 11, 3)
    except: 
        try:
            trendCurve = savgol_filter(trendCurve, 9, 3)
        except:
            try:
                trendCurve = savgol_filter(trendCurve, 7, 3)
            except: trendCurve = trendCurve

    maxs = []
    for j in range(1,trendCurve.shape[0]-1):
        if(trendCurve[j-1] < trendCurve[j] and trendCurve[j+1] < trendCurve[j]
          and signal[j+Q] and signal[j+Q] and trendCurve[j]>3):
            maxs.append(j)
    
    if(len(maxs) < 2):
        return 0
    
    length = len(maxs)
    for i in range(1,length):
        if (maxs[i]-maxs[i-1] >= 3):
            return 1
        
    return 0

def varClusters(templates, P, R):
    swaped = np.swapaxes(templates,1,0)
    
    # 1 cluster
    sumVar = np.sum(swaped[P:R].var(axis=1))
    
    # 2 clusters
    qrSignal = swaped[P:R]
    var = 0
    for i in range(0,qrSignal.shape[1]):
        numbers = qrSignal[i]
        kmeans = KMeans(n_clusters=2, random_state=0).fit(numbers.reshape(-1,1))
        labels = kmeans.labels_
        c0 = []
        c1 = []
        for k in range(0,numbers.shape[0]):
            if (labels[k] == 0):
                c0.append(numbers[k])
            else:
                c1.append(numbers[k])
        var = var + np.var(np.array(c0)) + np.var(np.array(c1))
      
    return [sumVar,var]

In [23]:
niciFeatures = []
j = 0

for i,row in tqdm(X_no3.iterrows(), total=X_no3.shape[0],position=0, leave=True):    
    # Appply sliding window
    var, start, end, templates, pr = shiftWindow(8,row)
    
    # find P,Q,R etc.
    swaped = np.swapaxes(templates,1,0)
    signal = []
    for i in range(0,180):
        signal.append(np.median(swaped[i]))
    signal = np.array(signal)
    P = findP(signal)
    Q = findQ(signal)
    R = findR(signal)
    S = findS(signal)
    T = findT(signal)
    Ts = findTstart(signal, T)
    Te = findTend(signal, T)
    STelbow = findSTelbow(signal,S)
    mins = QRTrendChange(signal, Q, R)
    var1, var2 =  varClusters(templates,P, R-3)
    
    a = [var, P, signal[P], Q, signal[Q], signal[R], S, signal[S], 
        T, signal[T], Ts, signal[Ts], Te, signal[Te],
        STelbow, signal[STelbow], mins, var1, var2]
    
    niciFeatures.append(a)
    
niciFeatures = np.array(niciFeatures)

100%|██████████| 4947/4947 [1:21:35<00:00,  1.01it/s]


#### Mo features

In [24]:
def classfeatures(signal):
    # For the preprocessed Signal cal sumVar
    signal2 = removeStartEnd(signal)
    ecgResults = ecg(signal2,sampling_rate=300, show=False)
    templates = np.swapaxes(ecgResults[4],1,0) #This is adjusted signal
    peakAmplitudes = []
    for k in ecgResults[2]:
        peakAmplitudes.append(ecgResults[1][k])
    amplitudeMedian = np.median(np.array(peakAmplitudes))
    scaledTemplates = templates/amplitudeMedian
    sumVar = np.sum(scaledTemplates.var(axis=1))
    
    # Feature 0
    nvar = sumVar # Scaled (by R-Median) Variance of the preprocessed ECG Signal
    
    
    # Get the Qs and Ss
    [Qtime, Qs] = getQsNormal(signal2,ecgResults[2]) # [QTime, QValue], get value&Amplitude of Q
    [Stime, Ss] = getSsNormal(signal2,ecgResults[2]) # [sTime, sValue], get value&Amplitude of S
    
    #Features more for finding Class 1:
    diffR = ecgResults[2]
    diffR = diffR[1:] - diffR[:(len(diffR)-1)]
    Rvar = np.var(diffR) #Variance of R timepoint-difference
    RIQA = np.quantile(diffR,0.75) - np.quantile(diffR,0.25) #IQA of R timepoint-difference
    diff = np.array(Qtime)
    diff = diff[1:] - diff[:(len(diff)-1)]
    QIQA = np.quantile(diff,0.75) - np.quantile(diff,0.25) #IQA of Q timepoint-difference
    diff = np.array(Stime)
    diff = diff[1:] - diff[:(len(diff)-1)]
    SIQA = np.quantile(diff,0.75) - np.quantile(diff,0.25) #IQA of S timepoint-difference
    
    #To eliminate X[22] bzw class 2 with Rpeak and high T that gets recognized as R -> Take difference of difference
    #-> If R-diff (in t) is big-small-big-small -> We find it this way
    diffofdiffR = np.absolute(diffR[1:] - diffR[:(len(diffR)-1)])
    diffofdiffRmed = np.median(diffofdiffR)
    
    #Include heartrate variability: Proposed in slide and maybe is better version of R-Variance
    if ecgResults[6].size == 0: #If ecg doesn't calculate a heartbeat, just set the variables to 0
        heartratevar = 0
        heartrateIQA = 0
        heartratemed = 80
    else:
        heartratevar = ecgResults[6].var(axis=0)
        heartrateIQA = np.quantile(ecgResults[6],0.75) - np.quantile(ecgResults[6],0.25)
        heartratemed = np.median(ecgResults[6])
    
    #Features more for differentiating Class 0&2:
    #Include robust amplitude(median) of Q,R,S of preprocessed signal (Because of slides)
    #Note that I leave Ramp, while I scale Qamp&Samp by Ramp to make it comparable across samples with diff. amplitudes
    qsTime, qsValue = getQs(ecgResults)
    ssTime, ssValue = getSs(ecgResults)
    Rs = ecgResults[1][ecgResults[2]]
    Ramp = np.median(Rs)
    Qamp = np.median(qsValue)/Ramp
    Samp = np.median(ssValue)/Ramp
    
    
    #Include the robust difference of Q-S time interval and Q-S amplitude(of preprocessed signal)
    QRS = np.array(ssTime) - np.array(qsTime)
    QRSmed = np.median(QRS) #Recommended in tutorial, median of distance from Q to S
    QSampdiff = np.array(qsValue) - np.array(ssValue) #amplitude of q-s (should be mildly positive)
    QSampdiffmed = np.median(QSampdiff)/Ramp #Robust Difference in amplitude of Q and S scaled by Ramp
    QSampdiffIQA = np.quantile(QSampdiff,0.9) - np.quantile(QSampdiff,0.1) #for Nr.3 where it varies
    
    #Find T as the maximum of values between S and next Q to find this anomaly in Class 2. 
    Rpeaks = ecgResults[2]
    Tpeaks = [None]*(np.shape(ecgResults[2])[0]-1)

    for i in range(np.shape(ecgResults[2])[0]-1): #Find maximum of values between S and Q of next heartbeat -> T
        if ssTime[i]>=qsTime[i+1]: #For 52 it has two R without a S inbetween -> gives error.
            if i == 0: #If q(t+1) and s(t) are the same for the first R peak -> Set to 0 bzw make it an outlier
                Tpeaks[i] = 0
            else:
                Tpeaks[i] = Tpeaks[i-1]
        else:       
            Tpeaks[i] = max(ecgResults[1][ssTime[i]:qsTime[i+1]]) 
    Tamp = np.median(Tpeaks)/Ramp
    
    #Compare size of S with size of T scaled by R
    STdiff = Tpeaks - np.array(ssValue[:np.shape(ecgResults[2])[0]-1])
    STdiffmed = np.median(STdiff)/Ramp
    
    #Most promising features in my head(w/o testing by ML): 
    #For finding Class 1: RIQA(combined with DiffofdiffRmed)
    #For finding Class 2: QSampdiffmed,Tamp

    return [RIQA, Rvar, diffofdiffRmed, QIQA, SIQA, heartratevar, heartrateIQA, 
            heartratemed, Ramp, Qamp, Samp, QRSmed, QSampdiffmed, QSampdiffIQA, Tamp, STdiffmed, nvar]
    

In [25]:
moFeatures = []

for i,row in tqdm(X_no3.iterrows(), total=X_no3.shape[0],position=0, leave=True): 
    moFeatures.append(classfeatures(row))
    
moFeatures = np.array(moFeatures)

100%|██████████| 4947/4947 [02:33<00:00, 32.32it/s]


#### HRV features

In [26]:
import warnings
warnings.filterwarnings('ignore')
import pyhrv.hrv as hrv
import pyhrv.tools as tools
import pyhrv.time_domain as td
import pyhrv.nonlinear as nl
import pyhrv.frequency_domain as fd


def featuresHRV(signal):
    res=[]
    ecgResults = ecg(signal,sampling_rate=300, show=False)
    rpeaks = ecgResults[2]
    nni = tools.nn_intervals(ecgResults[2])
    a = td.nni_parameters(nni,rpeaks)
    for el in a:
        res.append(el)
    b = td.nni_differences_parameters(nni,rpeaks)
    for el in b:
        res.append(el)
    c = td.hr_parameters(nni,rpeaks)
    for el in c:
        res.append(el)
    d = td.sdnn(nni,rpeaks)
    for el in d:
        res.append(el)
    e = td.rmssd(nni,rpeaks)
    for el in e:
        res.append(el)
    f = td.sdsd(nni,rpeaks)
    for el in f:
        res.append(el)  
    x = td.nnXX(nni,rpeaks,5)
    for el in x:
        res.append(el)

    y = td.nn50(nni,rpeaks)
    for el in y:
        res.append(el)

    z = td.nn20(nni,rpeaks)
    for el in z:
        res.append(el)

    return res


In [27]:
features_hrv = []

for i,row in tqdm(X_no3.iterrows(), total=X_no3.shape[0],position=0, leave=True): 
    features_hrv.append(featuresHRV(row))
    
features_hrv = np.array(features_hrv)

100%|██████████| 4947/4947 [02:39<00:00, 31.07it/s]


In [28]:
df_hrv = pd.DataFrame(np.array(features_hrv))
df_mo = pd.DataFrame(np.array(moFeatures))
df_nici = pd.DataFrame(np.array(niciFeatures))

df_hrv.to_csv('df_hrv_train_unscaled.csv',index=False)
df_mo.to_csv('df_mo_train_unscaled.csv',index=False)
df_nici.to_csv('df_nici_train_unscaled.csv',index=False)

In [29]:
scalerHRV = StandardScaler().fit(df_hrv)
df_hrv_scaled = pd.DataFrame(scalerHRV.transform(df_hrv))

scalerMo = StandardScaler().fit(df_mo)
df_mo_scaled = pd.DataFrame(scalerMo.transform(df_mo))

scalerNici = StandardScaler().fit(df_nici)
df_nici_scaled = pd.DataFrame(scalerNici.transform(df_nici))

In [30]:
df_hrv_scaled.to_csv('df_hrv_train.csv',index=False)
df_mo_scaled.to_csv('df_mo_train.csv',index=False)
df_nici_scaled.to_csv('df_nici_train.csv',index=False)

### Test features

In [40]:
X_t = pd.read_csv('X_test.csv')
X_t = X_t.drop(['id'],axis=1)
X_t = X_t.fillna(0.0)
X_t.at[3410,'x17978'] = 0.0
X_t = X_t.astype(float)

In [43]:
invs = []
fract = []
for i in range(0, X_t.shape[0]):
    var = onlyVar(X_t.iloc[i])
    varI = onlyVar(X_t.iloc[i]*(-1))
    if (varI < var):
        invs.append(-1)
        fract.append(var/varI)
    else:
        invs.append(1)
        fract.append(varI/var)

In [44]:
X_corr_t = X_t.mul(np.array(invs),axis=0)

In [45]:
for i in range(0,X_corr_t.shape[0]):
    signal = X_corr_t.iloc[i]
    if (i ==584):
        print(i)
    else:
        newsignal = delMessfehlerMinimum(signal,1.7,50)
        newsignal = delMessfehlerMaximum(newsignal,1.7,50)
        X_corr_t.iloc[i] = newsignal

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

#### Nici Features

In [48]:
niciFeatures_t = []

for i,row in tqdm(X_corr_t.iterrows(), total=X_corr_t.shape[0],position=0, leave=True):    
    # Appply sliding window
    var, start, end, templates, pr = shiftWindow(8,row)
    
    # find P,Q,R etc.
    swaped = np.swapaxes(templates,1,0)
    signal = []
    for i in range(0,180):
        signal.append(np.median(swaped[i]))
    signal = np.array(signal)
    P = findP(signal)
    Q = findQ(signal)
    R = findR(signal)
    S = findS(signal)
    T = findT(signal)
    Ts = findTstart(signal, T)
    Te = findTend(signal, T)
    STelbow = findSTelbow(signal,S)
    mins = QRTrendChange(signal, Q, R)
    var1, var2 =  varClusters(templates,P, R-3)
    
    a = [var, P, signal[P], Q, signal[Q], signal[R], S, signal[S], 
        T, signal[T], Ts, signal[Ts], Te, signal[Te],
        STelbow, signal[STelbow], mins, var1, var2]
    
    niciFeatures_t.append(a)
    
niciFeatures_t = np.array(niciFeatures_t)

100%|██████████| 3411/3411 [56:28<00:00,  1.01it/s]  


#### Mo features

In [49]:
moFeatures_t = []

for i,row in tqdm(X_corr_t.iterrows(), total=X_corr_t.shape[0],position=0, leave=True): 
    moFeatures_t.append(classfeatures(row))
    
moFeatures_t = np.array(moFeatures_t)

100%|██████████| 3411/3411 [01:46<00:00, 32.07it/s]


#### HRV features

In [50]:
features_hrv_t = []

for i,row in tqdm(X_corr_t.iterrows(), total=X_corr_t.shape[0],position=0, leave=True): 
    features_hrv_t.append(featuresHRV(row))
    
features_hrv_t = np.array(features_hrv_t)

100%|██████████| 3411/3411 [01:50<00:00, 30.96it/s]


In [51]:
df_hrv_t = pd.DataFrame(np.array(features_hrv_t))
df_mo_t = pd.DataFrame(np.array(moFeatures_t))
df_nici_t = pd.DataFrame(np.array(niciFeatures_t))

df_hrv_t.to_csv('df_hrv_test_unscaled.csv',index=False)
df_mo_t.to_csv('df_mo_test_unscaled.csv',index=False)
df_nici_t.to_csv('df_nici_test_unscaled.csv',index=False)

In [52]:
df_hrv_t_scaled = pd.DataFrame(scalerHRV.transform(df_hrv_t))
df_mo_t_scaled = pd.DataFrame(scalerMo.transform(df_mo_t))
df_nici_t_scaled = pd.DataFrame(scalerNici.transform(df_nici_t))

In [53]:
df_hrv_t_scaled.to_csv('df_hrv_test.csv',index=False)
df_mo_t_scaled.to_csv('df_mo_test.csv',index=False)
df_nici_t_scaled.to_csv('df_nici_test.csv',index=False)

## Nici + HRV

In [54]:
df_train = pd.concat([df_nici_scaled,df_hrv_scaled],axis=1)
df_test = pd.concat([df_nici_t_scaled,df_hrv_t_scaled],axis=1)

In [55]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
selector = SelectKBest(f_classif, k=20).fit(df_train, np.ravel(y_no3))
selector.scores_

array([3.89563507e+01, 9.02038419e+01, 1.08773866e+02, 4.92403167e+01,
       1.73871014e+00, 1.34266124e+00, 4.55441388e+01, 1.13933106e+01,
       8.97075982e+01, 9.41842654e+00, 1.84834986e+01, 7.18241907e+00,
       4.75485996e+00, 2.06613514e-01, 4.35442559e+01, 1.19724557e+01,
       2.33405493e+01, 2.40413212e+01, 2.16884142e+01, 1.01027402e+02,
       1.21234865e+02, 1.59642753e+02, 4.89577251e+01, 4.61710178e+02,
       4.02069981e+01, 2.25405392e+02, 2.79598638e+02, 9.93261103e+00,
       2.08422619e+01, 4.58997187e+01, 1.81560976e+02, 3.09294278e+02,
       1.66617658e+02, 7.16744991e+02, 4.96354999e+02, 7.78497036e+02,
       8.13402762e+02, 1.29409760e+03, 1.25073819e+03])

### SVM Poly (~ 76%)

In [56]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from pactools.grid_search import GridSearchCVProgressBar

In [57]:
warnings.filterwarnings('ignore')
# Create pipeline object with standard scaler and SVC estimator
pipe = Pipeline([('svm_poly', SVC(kernel='poly', random_state=0, class_weight = "balanced"))])
#######Polynomial Kernel Function#######
param_grid = {'svm_poly__C': [1] ,
              'svm_poly__degree': [3],
              'svm_poly__gamma': ['scale'],
              'svm_poly__coef0':[1]}

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro' )
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

2020-11-29 01:42:00.273894


/home/moritz/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best F1 score: 0.7768
Best parameters: {'svm_poly__C': 1, 'svm_poly__coef0': 1, 'svm_poly__degree': 3, 'svm_poly__gamma': 'scale'}
2020-11-29 01:42:02.141585


In [58]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svm_poly__C,param_svm_poly__coef0,param_svm_poly__degree,param_svm_poly__gamma,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.483921,0.011736,0.035753,0.000512,1,1,3,scale,"{'svm_poly__C': 1, 'svm_poly__coef0': 1, 'svm_...",0.769697,...,0.783838,0.771717,0.783838,0.775758,0.775304,0.755061,0.777328,0.77683,0.009851,1


In [59]:
clf_svm = SVC(kernel='poly',class_weight = "balanced")
clf_svm.fit(df_train, np.ravel(y_no3))
ypred = clf_svm.predict(df_test)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('svmPoly.csv',index=False)

### SVM RBF (~ 76%)

In [60]:
warnings.filterwarnings('ignore')
# Create pipeline object with standard scaler and SVC estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)), 
                     ('svm_rbf', SVC(kernel='rbf', random_state=0, class_weight = "balanced"))])
#######RBF Kernel Function#######
param_grid = {'selector__k': [10,39],
              'svm_rbf__C': [1], 
              'svm_rbf__gamma': ['scale'] }

# Run grid search
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro', verbose=10)
grid = grid.fit(df_train, np.ravel(y_no3))

print(datetime.datetime.now()) #computation time
print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   4 out of  20 | elapsed:    0.8s remaining:    3.0s
[Parallel(n_jobs=22)]: Done   7 out of  20 | elapsed:    0.9s remaining:    1.6s
[Parallel(n_jobs=22)]: Done  10 out of  20 | elapsed:    0.9s remaining:    0.9s
[Parallel(n_jobs=22)]: Done  13 out of  20 | elapsed:    1.0s remaining:    0.5s
[Parallel(n_jobs=22)]: Done  16 out of  20 | elapsed:    1.1s remaining:    0.3s
[Parallel(n_jobs=22)]: Done  20 out of  20 | elapsed:    1.3s finished


2020-11-29 01:42:05.064915
Best F1 score: 0.7778
Best parameters: {'selector__k': 39, 'svm_rbf__C': 1, 'svm_rbf__gamma': 'scale'}
2020-11-29 01:42:05.065070


In [61]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selector__k,param_svm_rbf__C,param_svm_rbf__gamma,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.401886,0.061911,0.024864,0.004319,10,1,scale,"{'selector__k': 10, 'svm_rbf__C': 1, 'svm_rbf_...",0.711111,0.717172,...,0.745455,0.749495,0.729293,0.721212,0.775304,0.672065,0.736842,0.732360,0.028088,2
1,0.698982,0.068089,0.055616,0.008217,39,1,scale,"{'selector__k': 39, 'svm_rbf__C': 1, 'svm_rbf_...",0.767677,0.779798,...,0.781818,0.765657,0.783838,0.787879,0.773279,0.769231,0.785425,0.777844,0.007724,1


In [62]:
k = grid.best_params_['selector__k']
SelectorSVM = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorSVM.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorSVM.transform(df_test))

clf_svm = SVC(kernel='rbf',class_weight = "balanced")
clf_svm.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_svm.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('svmRBF.csv',index=False)

### Random Forest classifier (~80%)

In [63]:
# Create pipeline object with standard scaler and RF estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)), 
                     ('RF', RandomForestClassifier(random_state=0))])
#######RF Kernel Function#######
param_grid = {'selector__k': [10,20,30,39],
              'RF__n_estimators': [100,500,1000], 
              'RF__max_depth': [5,10,15,20],
              'RF__class_weight': [None,'balanced']
             }

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro',verbose=10 )
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())


2020-11-29 01:42:05.969928
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=22)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=22)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=22)]: Done  41 tasks      | elapsed:    2.7s
[Parallel(n_jobs=22)]: Done  54 tasks      | elapsed:    3.7s
[Parallel(n_jobs=22)]: Done  69 tasks      | elapsed:    6.1s
[Parallel(n_jobs=22)]: Done  84 tasks      | elapsed:    8.5s
[Parallel(n_jobs=22)]: Done 101 tasks      | elapsed:   13.2s
[Parallel(n_jobs=22)]: Done 118 tasks      | elapsed:   15.0s
[Parallel(n_jobs=22)]: Done 137 tasks      | elapsed:   16.5s
[Parallel(n_jobs=22)]: Done 156 tasks      | elapsed:   19.1s
[Parallel(n_jobs=22)]: Done 177 tasks      | elapsed:   23.2s
[Parallel(n_jobs=22)]: Done 198 tasks      | elapsed:   28.6s
[Parallel(n_jobs=22)]: Done 221 tasks      | elapsed:   37.2s
[Parallel(n_jobs=22)]: Done 244 tasks      | elapsed:  

Best F1 score: 0.8136
Best parameters: {'RF__class_weight': None, 'RF__max_depth': 20, 'RF__n_estimators': 100, 'selector__k': 30}
2020-11-29 01:45:53.520701


In [64]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_RF__class_weight,param_RF__max_depth,param_RF__n_estimators,param_selector__k,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.409740,0.010586,0.012372,0.001445,None,5,100,10,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.771717,...,0.755556,0.755556,0.759596,0.757576,0.757085,0.710526,0.769231,0.757220,0.017790,84
1,0.513453,0.022473,0.013154,0.001039,None,5,100,20,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.781818,...,0.787879,0.779798,0.773737,0.773737,0.775304,0.775304,0.787449,0.780068,0.007774,63
2,0.583532,0.028281,0.012252,0.001665,None,5,100,30,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.781818,...,0.791919,0.781818,0.767677,0.781818,0.779352,0.775304,0.779352,0.780674,0.008670,61
3,0.723200,0.030459,0.012853,0.001514,None,5,100,39,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.783838,...,0.783838,0.779798,0.771717,0.777778,0.773279,0.775304,0.785425,0.780269,0.008171,62
4,2.019862,0.044405,0.051508,0.004404,None,5,500,10,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.773737,...,0.757576,0.751515,0.755556,0.755556,0.761134,0.712551,0.769231,0.757019,0.017288,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,7.984330,0.192597,0.071774,0.005335,balanced,20,500,39,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.816162,...,0.818182,0.800000,0.800000,0.822222,0.811741,0.795547,0.819838,0.812410,0.010986,3
92,7.996780,0.180704,0.142144,0.008189,balanced,20,1000,10,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.783838,...,0.765657,0.781818,0.761616,0.765657,0.771255,0.722672,0.761134,0.765304,0.017889,76
93,10.745809,0.196730,0.135334,0.010171,balanced,20,1000,20,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.797980,...,0.806061,0.806061,0.800000,0.797980,0.803644,0.793522,0.799595,0.804929,0.016487,28
94,12.500927,0.299007,0.125632,0.012108,balanced,20,1000,30,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.814141,...,0.818182,0.793939,0.793939,0.806061,0.807692,0.793522,0.809717,0.806144,0.011891,24


In [65]:
k = grid.best_params_['selector__k']
class_weight = grid.best_params_['RF__class_weight']
n_estimators = grid.best_params_['RF__n_estimators']
max_depth = grid.best_params_['RF__max_depth']


SelectorRF = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorRF.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorRF.transform(df_test))

clf_rf = RandomForestClassifier(class_weight=class_weight, n_estimators=n_estimators, max_depth=max_depth)
clf_rf.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_rf.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('randomForest.csv',index=False)

### Gradientboosting (~80%)

In [66]:
# Create pipeline object with standard scaler and Adaboost estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)),
                     ('GBoost', GradientBoostingClassifier(random_state=0))])
#######Adaboost Kernel Function#######
param_grid = {'selector__k': [10,20,30,39],
              'GBoost__n_estimators': [100,500,750,1000], 
              'GBoost__learning_rate': [0.1,1],
              'GBoost__max_depth': [5,10,15,20]
             }

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro',verbose=10)
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

2020-11-29 01:45:54.766523
Fitting 10 folds for each of 128 candidates, totalling 1280 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   6 tasks      | elapsed:    4.4s
[Parallel(n_jobs=22)]: Done  17 tasks      | elapsed:    8.5s
[Parallel(n_jobs=22)]: Done  28 tasks      | elapsed:   16.4s
[Parallel(n_jobs=22)]: Done  41 tasks      | elapsed:   28.5s
[Parallel(n_jobs=22)]: Done  54 tasks      | elapsed:  1.1min
[Parallel(n_jobs=22)]: Done  69 tasks      | elapsed:  1.6min
[Parallel(n_jobs=22)]: Done  84 tasks      | elapsed:  2.4min
[Parallel(n_jobs=22)]: Done 101 tasks      | elapsed:  3.6min
[Parallel(n_jobs=22)]: Done 118 tasks      | elapsed:  4.5min
[Parallel(n_jobs=22)]: Done 137 tasks      | elapsed:  5.7min
[Parallel(n_jobs=22)]: Done 156 tasks      | elapsed:  6.8min
[Parallel(n_jobs=22)]: Done 177 tasks      | elapsed:  7.4min
[Parallel(n_jobs=22)]: Done 198 tasks      | elapsed:  8.3min
[Parallel(n_jobs=22)]: Done 221 tasks      | elapsed: 10.4min
[Parallel(n_jobs=22)]: Done 244 tasks      | elapsed: 1

Best F1 score: 0.8217
Best parameters: {'GBoost__learning_rate': 0.1, 'GBoost__max_depth': 5, 'GBoost__n_estimators': 1000, 'selector__k': 39}
2020-11-29 04:28:31.426855


In [67]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_GBoost__learning_rate,param_GBoost__max_depth,param_GBoost__n_estimators,param_selector__k,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,4.282498,0.128145,0.005460,0.000670,0.1,5,100,10,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.757576,...,0.769697,0.777778,0.761616,0.767677,0.769231,0.738866,0.751012,0.765507,0.013783,82
1,8.382379,0.222854,0.005711,0.000383,0.1,5,100,20,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.806061,...,0.806061,0.806061,0.804040,0.804040,0.789474,0.827935,0.793522,0.808568,0.013458,23
2,11.783654,0.334157,0.005652,0.000349,0.1,5,100,30,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.824242,...,0.806061,0.810101,0.812121,0.802020,0.783401,0.842105,0.803644,0.813218,0.016999,11
3,16.846471,0.217535,0.005634,0.000410,0.1,5,100,39,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.824242,...,0.830303,0.808081,0.810101,0.818182,0.787449,0.825911,0.805668,0.815236,0.015308,7
4,21.314631,0.718843,0.014622,0.001605,0.1,5,500,10,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.761616,...,0.761616,0.767677,0.759596,0.739394,0.775304,0.710526,0.726721,0.753174,0.021429,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,490.722412,6.758321,0.186940,0.005036,1,20,750,39,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.684848,...,0.725253,0.719192,0.733333,0.707071,0.684211,0.696356,0.692308,0.704055,0.016391,123
124,231.933314,6.190729,0.220644,0.009145,1,20,1000,10,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.707071,...,0.682828,0.709091,0.676768,0.656566,0.690283,0.655870,0.714575,0.690719,0.022066,125
125,366.368977,8.374371,0.225891,0.014063,1,20,1000,20,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.713131,...,0.664646,0.725253,0.678788,0.640404,0.663968,0.730769,0.751012,0.699423,0.034133,124
126,499.796656,11.581218,0.230856,0.012695,1,20,1000,30,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.684848,...,0.658586,0.589899,0.624242,0.654545,0.684211,0.665992,0.635628,0.655351,0.029198,128


In [68]:
k = grid.best_params_['selector__k']
n_estimators = grid.best_params_['GBoost__n_estimators']
learning_rate = grid.best_params_['GBoost__learning_rate']
max_depth = grid.best_params_['GBoost__max_depth']


SelectorRF = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorRF.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorRF.transform(df_test))

clf_gb = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
clf_gb.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_gb.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('gradientBoost.csv',index=False)

## Nici + Mo + HRV

In [69]:
df_train = pd.concat([df_nici_scaled,df_mo_scaled,df_hrv_scaled],axis=1)
df_test = pd.concat([df_nici_t_scaled,df_mo_t_scaled,df_hrv_t_scaled],axis=1)

In [70]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
selector = SelectKBest(f_classif, k=20).fit(df_train, np.ravel(y_no3))
selector.scores_

array([3.89563507e+01, 9.02038419e+01, 1.08773866e+02, 4.92403167e+01,
       1.73871014e+00, 1.34266124e+00, 4.55441388e+01, 1.13933106e+01,
       8.97075982e+01, 9.41842654e+00, 1.84834986e+01, 7.18241907e+00,
       4.75485996e+00, 2.06613514e-01, 4.35442559e+01, 1.19724557e+01,
       2.33405493e+01, 2.40413212e+01, 2.16884142e+01, 3.56090851e+02,
       4.62278699e+00, 4.88670709e+02, 4.08891977e+02, 3.45625889e+02,
       1.04464223e+02, 3.36078249e+02, 3.70776319e+02, 9.57304301e+00,
       1.13998231e+00, 7.99999348e+00, 7.59227611e+01, 3.33816936e+00,
       4.08075374e+01, 1.06746963e+01, 3.87278351e+00, 4.15995366e+01,
       1.01027402e+02, 1.21234865e+02, 1.59642753e+02, 4.89577251e+01,
       4.61710178e+02, 4.02069981e+01, 2.25405392e+02, 2.79598638e+02,
       9.93261103e+00, 2.08422619e+01, 4.58997187e+01, 1.81560976e+02,
       3.09294278e+02, 1.66617658e+02, 7.16744991e+02, 4.96354999e+02,
       7.78497036e+02, 8.13402762e+02, 1.29409760e+03, 1.25073819e+03])

### SVM Poly (~ 76%)

In [71]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from pactools.grid_search import GridSearchCVProgressBar

In [72]:
warnings.filterwarnings('ignore')
# Create pipeline object with standard scaler and SVC estimator
pipe = Pipeline([('svm_poly', SVC(kernel='poly', random_state=0, class_weight = "balanced"))])
#######Polynomial Kernel Function#######
param_grid = {'svm_poly__C': [1] ,
              'svm_poly__degree': [3],
              'svm_poly__gamma': ['scale'],
              'svm_poly__coef0':[1]}

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro' )
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

2020-11-29 04:30:58.105852
Best F1 score: 0.7851
Best parameters: {'svm_poly__C': 1, 'svm_poly__coef0': 1, 'svm_poly__degree': 3, 'svm_poly__gamma': 'scale'}
2020-11-29 04:31:00.185370


In [73]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svm_poly__C,param_svm_poly__coef0,param_svm_poly__degree,param_svm_poly__gamma,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.568075,0.01327,0.047142,0.000451,1,1,3,scale,"{'svm_poly__C': 1, 'svm_poly__coef0': 1, 'svm_...",0.8,...,0.8,0.779798,0.773737,0.783838,0.789474,0.767206,0.783401,0.785119,0.010011,1


In [74]:
clf_svm = SVC(kernel='poly',class_weight = "balanced")
clf_svm.fit(df_train, np.ravel(y_no3))
ypred = clf_svm.predict(df_test)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('svmPolyAll.csv',index=False)

### SVM RBF (~ 76%)

In [75]:
warnings.filterwarnings('ignore')
# Create pipeline object with standard scaler and SVC estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)), 
                     ('svm_rbf', SVC(kernel='rbf', random_state=0, class_weight = "balanced"))])
#######RBF Kernel Function#######
param_grid = {'selector__k': [10,39],
              'svm_rbf__C': [1], 
              'svm_rbf__gamma': ['scale'] }

# Run grid search
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro', verbose=10)
grid = grid.fit(df_train, np.ravel(y_no3))

print(datetime.datetime.now()) #computation time
print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   4 out of  20 | elapsed:    0.7s remaining:    2.9s
[Parallel(n_jobs=22)]: Done   7 out of  20 | elapsed:    0.8s remaining:    1.4s
[Parallel(n_jobs=22)]: Done  10 out of  20 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=22)]: Done  13 out of  20 | elapsed:    1.0s remaining:    0.5s
[Parallel(n_jobs=22)]: Done  16 out of  20 | elapsed:    1.0s remaining:    0.2s
[Parallel(n_jobs=22)]: Done  20 out of  20 | elapsed:    1.2s finished


2020-11-29 04:31:03.410322
Best F1 score: 0.7839
Best parameters: {'selector__k': 39, 'svm_rbf__C': 1, 'svm_rbf__gamma': 'scale'}
2020-11-29 04:31:03.410487


In [76]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selector__k,param_svm_rbf__C,param_svm_rbf__gamma,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.396293,0.035809,0.022376,0.000914,10,1,scale,"{'selector__k': 10, 'svm_rbf__C': 1, 'svm_rbf_...",0.719192,0.713131,...,0.743434,0.733333,0.676768,0.656566,0.748988,0.674089,0.728745,0.715788,0.033792,2
1,0.622175,0.052348,0.048848,0.007523,39,1,scale,"{'selector__k': 39, 'svm_rbf__C': 1, 'svm_rbf_...",0.814141,0.771717,...,0.795960,0.769697,0.779798,0.779798,0.785425,0.769231,0.781377,0.783906,0.013160,1


In [77]:
k = grid.best_params_['selector__k']
SelectorSVM = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorSVM.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorSVM.transform(df_test))

clf_svm = SVC(kernel='rbf',class_weight = "balanced")
clf_svm.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_svm.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('svmRBFAll.csv',index=False)

### Random Forest classifier (~80%)

In [78]:
# Create pipeline object with standard scaler and RF estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)), 
                     ('RF', RandomForestClassifier(random_state=0))])
#######RF Kernel Function#######
param_grid = {'selector__k': [10,20,30,39],
              'RF__n_estimators': [100,500,1000], 
              'RF__max_depth': [5,10,15,20],
              'RF__class_weight': [None,'balanced']
             }

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro',verbose=10 )
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())


2020-11-29 04:31:04.304002
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=22)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=22)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=22)]: Done  41 tasks      | elapsed:    2.6s
[Parallel(n_jobs=22)]: Done  54 tasks      | elapsed:    3.9s
[Parallel(n_jobs=22)]: Done  69 tasks      | elapsed:    5.9s
[Parallel(n_jobs=22)]: Done  84 tasks      | elapsed:    8.4s
[Parallel(n_jobs=22)]: Done 101 tasks      | elapsed:   13.4s
[Parallel(n_jobs=22)]: Done 118 tasks      | elapsed:   15.0s
[Parallel(n_jobs=22)]: Done 137 tasks      | elapsed:   16.4s
[Parallel(n_jobs=22)]: Done 156 tasks      | elapsed:   18.8s
[Parallel(n_jobs=22)]: Done 177 tasks      | elapsed:   22.8s
[Parallel(n_jobs=22)]: Done 198 tasks      | elapsed:   28.2s
[Parallel(n_jobs=22)]: Done 221 tasks      | elapsed:   37.6s
[Parallel(n_jobs=22)]: Done 244 tasks      | elapsed:  

Best F1 score: 0.8219
Best parameters: {'RF__class_weight': None, 'RF__max_depth': 20, 'RF__n_estimators': 1000, 'selector__k': 39}
2020-11-29 04:35:04.051036


In [79]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_RF__class_weight,param_RF__max_depth,param_RF__n_estimators,param_selector__k,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.383710,0.015944,0.011985,0.001302,None,5,100,10,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.769697,...,0.777778,0.767677,0.723232,0.717172,0.767206,0.714575,0.783401,0.753589,0.025380,84
1,0.543163,0.010269,0.013047,0.000987,None,5,100,20,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.787879,...,0.779798,0.767677,0.781818,0.787879,0.781377,0.751012,0.775304,0.776426,0.012049,63
2,0.587240,0.034651,0.011307,0.001122,None,5,100,30,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.791919,...,0.789899,0.791919,0.777778,0.787879,0.777328,0.757085,0.791498,0.783702,0.013643,60
3,0.705447,0.007410,0.012996,0.000944,None,5,100,39,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.783838,...,0.789899,0.787879,0.785859,0.793939,0.779352,0.765182,0.789474,0.783906,0.009279,59
4,1.860906,0.094310,0.050125,0.003719,None,5,500,10,"{'RF__class_weight': None, 'RF__max_depth': 5,...",0.769697,...,0.777778,0.763636,0.725253,0.717172,0.769231,0.714575,0.777328,0.753184,0.023875,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,7.740774,0.178633,0.071495,0.002320,balanced,20,500,39,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.826263,...,0.816162,0.810101,0.800000,0.816162,0.825911,0.793522,0.809717,0.814026,0.014608,11
92,7.527106,0.221206,0.141631,0.016207,balanced,20,1000,10,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.769697,...,0.769697,0.779798,0.737374,0.721212,0.773279,0.718623,0.771255,0.758235,0.023237,77
93,11.096777,0.176352,0.136789,0.010244,balanced,20,1000,20,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.789899,...,0.795960,0.791919,0.791919,0.785859,0.801619,0.767206,0.783401,0.789566,0.012035,50
94,13.065811,0.427069,0.130898,0.012667,balanced,20,1000,30,"{'RF__class_weight': 'balanced', 'RF__max_dept...",0.802020,...,0.808081,0.808081,0.802020,0.802020,0.819838,0.793522,0.811741,0.808369,0.012220,26


In [80]:
k = grid.best_params_['selector__k']
class_weight = grid.best_params_['RF__class_weight']
n_estimators = grid.best_params_['RF__n_estimators']
max_depth = grid.best_params_['RF__max_depth']


SelectorRF = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorRF.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorRF.transform(df_test))

clf_rf = RandomForestClassifier(class_weight=class_weight, n_estimators=n_estimators, max_depth=max_depth)
clf_rf.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_rf.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('randomForestAll.csv',index=False)

### Gradientboosting (~80%)

In [81]:
# Create pipeline object with standard scaler and Adaboost estimator
pipe = Pipeline([('selector', SelectKBest(f_classif)),
                     ('GBoost', GradientBoostingClassifier(random_state=0))])
#######Adaboost Kernel Function#######
param_grid = {'selector__k': [10,20,30,39],
              'GBoost__n_estimators': [100,500,750,1000], 
              'GBoost__learning_rate': [0.1,1],
              'GBoost__max_depth': [5,10,15,20]
             }

# Run grid search
print(datetime.datetime.now()) #computation time
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, n_jobs=22, scoring = 'f1_micro',verbose=10)
grid = grid.fit(df_train, np.ravel(y_no3))

print('Best F1 score: {:.4f}'.format(grid.best_score_))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

2020-11-29 04:35:18.936074
Fitting 10 folds for each of 128 candidates, totalling 1280 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done   6 tasks      | elapsed:    3.9s
[Parallel(n_jobs=22)]: Done  17 tasks      | elapsed:    9.0s
[Parallel(n_jobs=22)]: Done  28 tasks      | elapsed:   16.3s
[Parallel(n_jobs=22)]: Done  41 tasks      | elapsed:   28.8s
[Parallel(n_jobs=22)]: Done  54 tasks      | elapsed:  1.1min
[Parallel(n_jobs=22)]: Done  69 tasks      | elapsed:  1.6min
[Parallel(n_jobs=22)]: Done  84 tasks      | elapsed:  2.4min
[Parallel(n_jobs=22)]: Done 101 tasks      | elapsed:  3.5min
[Parallel(n_jobs=22)]: Done 118 tasks      | elapsed:  4.4min
[Parallel(n_jobs=22)]: Done 137 tasks      | elapsed:  5.7min
[Parallel(n_jobs=22)]: Done 156 tasks      | elapsed:  6.9min
[Parallel(n_jobs=22)]: Done 177 tasks      | elapsed:  7.5min
[Parallel(n_jobs=22)]: Done 198 tasks      | elapsed:  8.3min
[Parallel(n_jobs=22)]: Done 221 tasks      | elapsed: 10.4min
[Parallel(n_jobs=22)]: Done 244 tasks      | elapsed: 1

Best F1 score: 0.8237
Best parameters: {'GBoost__learning_rate': 0.1, 'GBoost__max_depth': 5, 'GBoost__n_estimators': 500, 'selector__k': 39}
2020-11-29 07:16:44.448096


In [82]:
warnings.filterwarnings('ignore')
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_GBoost__learning_rate,param_GBoost__max_depth,param_GBoost__n_estimators,param_selector__k,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,3.795386,0.218057,0.005568,0.000120,0.1,5,100,10,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.755556,...,0.763636,0.789899,0.709091,0.709091,0.783401,0.700405,0.761134,0.751363,0.031864,87
1,8.926508,0.128100,0.005766,0.000086,0.1,5,100,20,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.783838,...,0.804040,0.793939,0.771717,0.787879,0.819838,0.791498,0.779352,0.794018,0.015331,49
2,12.370232,0.370115,0.005719,0.000356,0.1,5,100,30,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.820202,...,0.816162,0.816162,0.816162,0.808081,0.805668,0.815789,0.815789,0.816654,0.011000,10
3,15.945668,0.482658,0.005877,0.000074,0.1,5,100,39,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.832323,...,0.826263,0.838384,0.806061,0.810101,0.803644,0.813765,0.809717,0.821904,0.017958,4
4,19.099640,0.610649,0.015652,0.000176,0.1,5,500,10,"{'GBoost__learning_rate': 0.1, 'GBoost__max_de...",0.737374,...,0.745455,0.781818,0.711111,0.694949,0.769231,0.682186,0.757085,0.740446,0.031955,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,478.594901,6.560956,0.179682,0.008190,1,20,750,39,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.743434,...,0.713131,0.713131,0.709091,0.721212,0.690283,0.700405,0.730769,0.715782,0.022538,117
124,225.529424,9.457005,0.222768,0.011161,1,20,1000,10,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.705051,...,0.715152,0.678788,0.610101,0.692929,0.706478,0.659919,0.698381,0.681023,0.029163,128
125,379.976103,8.041201,0.221689,0.012406,1,20,1000,20,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.723232,...,0.731313,0.703030,0.688889,0.735354,0.672065,0.653846,0.678138,0.687476,0.035036,127
126,511.460917,5.367432,0.222046,0.011695,1,20,1000,30,"{'GBoost__learning_rate': 1, 'GBoost__max_dept...",0.741414,...,0.612121,0.696970,0.670707,0.735354,0.631579,0.696356,0.686235,0.687882,0.040103,126


In [83]:
k = grid.best_params_['selector__k']
n_estimators = grid.best_params_['GBoost__n_estimators']
learning_rate = grid.best_params_['GBoost__learning_rate']
max_depth = grid.best_params_['GBoost__max_depth']


SelectorRF = SelectKBest(f_classif, k=k)
df_train_kbest = pd.DataFrame(SelectorRF.fit_transform(df_train, y_no3))
df_test_kbest = pd.DataFrame(SelectorRF.transform(df_test))

clf_gb = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
clf_gb.fit(df_train_kbest, np.ravel(y_no3))
ypred = clf_gb.predict(df_test_kbest)

solution = pd.read_csv('sample.csv')
solution['y'] = ypred
solution.to_csv('gradientBoostAll.csv',index=False)

## Include class 3

In [1]:
nicilist1 = [2,60,73,122,199,205,363,391,396,440,766,770,783,852,1008,1020,1142,1200,1314,1498,1527,1584,1898,2113,2765,2857,2882,3068,3098,3100,3318]
nicilist2 = [15,44,160,189,203,219,239,256,273,364,385,445,476,505,585,657,732,841,873,877,880,933,977,1034,1079,1501,1596,1612,1658,1751,1819,1857,2007,2014,2058,2067,2070,2110,2140,2234,2253,2373,2387,2478,2507,2513,2627,2657,2696,2806,2820,2953,3016,3022,3039,3174,3304,3317]
nicilist3 = [636,809,1034,1417,1421,1473,1672,1675,2251,2301,2728,2844,3207,3308]

## Prepare CSV file

In [14]:
def createClass3files(ypred, filename):
    # Nicilist1
    ypred = ypred
    for i in range(0,np.shape(ypred)[0]):
        if i in nicilist1:
            ypred[i] = 3
    
    solution = pd.read_csv('sample.csv')
    solution['y'] = ypred
    name = 'abgabe_' + filename + '1' + '.csv'
    solution.to_csv(name,index=False)
    
    # Nicilist2
    ypred = ypred
    for i in range(0,np.shape(ypred)[0]):
        if i in nicilist1 or i in nicilist2:
            ypred[i] = 3

    solution = pd.read_csv('sample.csv')
    solution['y'] = ypred
    name = 'abgabe_' + filename + '2' + '.csv'
    solution.to_csv(name,index=False)
    
    #Nicilist3
    ypred = ypred
    for i in range(0,np.shape(ypred)[0]):
        if i in nicilist1 or i in nicilist2 or i in nicilist3:
            ypred[i] = 3

    solution = pd.read_csv('sample.csv')
    solution['y'] = ypred
    name = 'abgabe_' + filename + '3' + '.csv'
    solution.to_csv(name,index=False)

In [15]:
# Nici + HRV
ypred = pd.read_csv('svmPoly.csv')
createClass3files(ypred['y'],'svmPoly')

ypred = pd.read_csv('svmRBF.csv')
createClass3files(ypred['y'],'svmRBF')

ypred = pd.read_csv('randomForest.csv')
createClass3files(ypred['y'],'randomForest')

ypred = pd.read_csv('gradientBoost.csv')
createClass3files(ypred['y'],'gradientBoost')

# Nici + Mo + HRV
ypred = pd.read_csv('svmPolyAll.csv')
createClass3files(ypred['y'],'svmPolyAll')

ypred = pd.read_csv('svmRBFAll.csv')
createClass3files(ypred['y'],'svmRBFAll')

ypred = pd.read_csv('randomForestAll.csv')
createClass3files(ypred['y'],'randomForestAll')

ypred = pd.read_csv('gradientBoostAll.csv')
createClass3files(ypred['y'],'gradientBoostAll')